# Task 3a. - Streaming Data Visualisation

In [ ]:
from time import sleep
from kafka import KafkaConsumer
import datetime as dt
import matplotlib.pyplot as plt
import json

# this line is needed for the inline display of graphs in Jupyter Notebook
%matplotlib notebook

topic = 'climate_data'

def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=10000, # stop iteration if no message after 10 sec
                                   auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

def init_plots():
    try:
        width = 9.5
        height = 6
        fig = plt.figure(figsize=(width,height)) # create new figure
        ax = fig.add_subplot(111) # adding the subplot axes to the given grid position
        fig.suptitle('Real Time Uniform Stream Air Temperature With Interesting Points') # giving figure a title
        ax.set_xlabel('Arrival Time')
        ax.set_ylabel('Air Temperature Celcius')
        ax.set_ylim(0,60) 
        ax.set_yticks([0,10,20,30,40,50,60])
        fig.show() # displaying the figure
        fig.canvas.draw() # drawing on the canvas
        return fig, ax
    except Exception as ex:
        print(str(ex))
        
def annotate_max(x, y, ax = None):
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = 'Max: Time={}, Value={}'.format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax, ymax+5), arrowprops=dict(facecolor='red', shrink=0.05),)
    
def annotate_min(x, y, ax = None):
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = 'Min: Time={}, Value={}'.format(xmin, ymin)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmin, ymin), xytext=(xmin, ymin+5), arrowprops=dict(facecolor='orange', shrink=0.05),)
    
def consume_messages(consumer, fig, ax):
    try:
        # container for x and y values
        x, y = [], []
        for message in consumer:
            data = json.loads(message.value.decode('ascii'))
            x.append(dt.datetime.now().strftime("%X")) 
            y.append(float(data['air_temperature_celcius']))
            # we start plotting only when we have 10 data points
            if len(y) > 10:
                ax.clear()
                ax.plot(x, y)
                ax.set_xlabel('Arrival Time')
                ax.set_ylabel('Air Temperature Celcius')
                ax.set_ylim(0,60) 
                ax.set_yticks([0,10,20,30,40,50,60])
                annotate_max(x,y)
                annotate_min(x,y)
                fig.canvas.draw()
                x.pop(0) # removing the item in the first position
                y.pop(0)
        plt.close('all')
    except Exception as ex:
        print(str(ex))
    
if __name__ == '__main__':
    consumer = connect_kafka_consumer()
    fig, ax = init_plots()
    consume_messages(consumer, fig, ax)

# Task 3b. Static Data Visualisation
## Task 3bi.

In [3]:
from pymongo import MongoClient

client = MongoClient()
db = client.fit5148_assignment_db
taskc = db.taskc

In [ ]:
import matplotlib.pyplot as plt
# to get plot in jupyter notebook
%matplotlib notebook
# initialise empty list to store time and fire count for plotting
time_fire = []
created_time = []
fire_count = []
# aggregated results of number of fires across different arrival time
results = taskc.aggregate([{"$unwind":"$hotspots"},{"$group":{"_id": "$arrival_time",\
                                                                "number_of_fires":{"$sum":1}}},\
                                                                {"$sort":{"number_of_fires":-1}},\
                                                                {"$limit":10}])

# add arrival time and fire count to the list
for each in results:
    time_fire.append((each['_id'],each['number_of_fires']))
    
# sort the time_fire according to arrival_time
time_fire = sorted(time_fire)

# break the list into two after sorting for plotting
for i in range(0,len(time_fire)):
    created_time.append(list(time_fire[i])[0])
    fire_count.append(list(time_fire[i])[1])

# plot the bar chart
plt.figure(figsize=(9,5))
plt.bar(created_time, fire_count)
plt.title("Top 10 Number of Fires")
plt.xlabel("Time")
plt.ylabel("Number of Fires")
plt.xticks(size=8)

## Task 3bii.

In [5]:
import gmplot
import webbrowser
# latitude and longitude of melbourne city
gmap = gmplot.GoogleMapPlotter(-37.812015244225677, 144.951471202974, 7)

# initialise empty list to lat, lon and rest of attributes for plotting
lat = []
lon = []
humidity = []
air_temp = []
surf_temp = []
confidence = []
# aggregated hotspots to the necessary attributes
results = taskc.aggregate([{"$unwind":"$hotspots"}, {"$project":{"_id":0,"air_temperature_celcius":1,\
                                            "relative_humidity":1, "hotspots.surface_temperature_celcius":1,\
                                            "hotspots.confidence":1, "hotspots.hotspot_latitude":1,\
                                            "hotspots.hotspot_longitude":1}}])

# Looping through all the data
for each in results:
    lat.append(each['hotspots']['hotspot_latitude'])
    lon.append(each['hotspots']['hotspot_longitude'])
    humidity.append(each['relative_humidity'])
    air_temp.append(each['air_temperature_celcius'])
    surf_temp.append(each['hotspots']['surface_temperature_celcius'])
    confidence.append(each['hotspots']['confidence'])

In [ ]:
# Plotting the points on the map
gmap.scatter(lat, lon,'#FF4500', size=3, marker=True)
# Adding in tooltip information
for i in range (0,len(lat)):
    temp_tooltip = 'surface temp: ' + str(surf_temp[i]) + ', confidence: ' + str(confidence[i]) + ', air temp: ' +\
                    str(air_temp[i]) + ', relative humidity: ' + str(humidity[i])
    gmap.marker(lat[i], lon[i], '#FF4500', title = temp_tooltip)

# Drawing the map
gmap.draw("ReportFire.html")
webbrowser.open_new("ReportFire.html")